In [1]:
# importing libraries
import numpy as np
import pandas as pd

In [2]:
# disable warning
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# reading the data frame
df1 = pd.read_csv('data_sets/january-2017.csv')
df2 = pd.read_csv('data_sets/february-2017.csv') 
df3 = pd.read_csv('data_sets/march-2017.csv')
df4 = pd.read_csv('data_sets/april-2017.csv') 
df5 = pd.read_csv('data_sets/may-2017.csv') 
df6 = pd.read_csv('data_sets/june-2017.csv') 
df7 = pd.read_csv('data_sets/july-2017.csv') 
df8 = pd.read_csv('data_sets/august-2017.csv') 
df9 = pd.read_csv('data_sets/september-2017.csv') 
df10 = pd.read_csv('data_sets/october-2017.csv') 
df11 = pd.read_csv('data_sets/november-2017.csv') 
df12 = pd.read_csv('data_sets/december-2017.csv') 

pd.set_option('display.max_rows', df.shape[0]+1)

In [4]:
# conbine the dataframe into one year
df = pd.concat([df1,df2,df3,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12])

In [5]:
# Dropping the columns that has more more than 10% of the data missing
for col in df:
    if df[col].isnull().sum() > 933:
        df.drop(col, axis=1, inplace=True)

In [6]:
# Dropping the columns with missing data (only take the sensors with temperature,humidity, pressure, pm1, pm25, and pm 10 )
# create a list of the first 3 digits of the col names
col_front = []
for col in df:
    col_front.append(col[:3])

In [7]:
# see which one has missing cols
pd.value_counts(col_front) < 6
pd.value_counts(col_front).index.tolist()
# create a list of all the sensors with missing columns
to_del = ['196','183', '176','180','184','177','185','181' '201','179','172','147']

In [8]:
# dropping the to_del from df
for col in df:
    if col[:3] in to_del:
        df.drop(col,axis=1,inplace=True)

In [9]:
# see which one is the remaining cols
pd.value_counts(col_front) == 6
pd.value_counts(col_front).index.tolist()
# create a list of the remaining cols
to_add = ['215','226',
 '171',
 '223',
 '208',
 '189',
 '220',
 '170',
 '169',
 '221',
 '219',
 '222',
 '225',
 '202',
 '173',
 '263',
 '212',
 '214',
 '210',
 '192',
 '209',
 '194',
 '218',
 '228',
 '204']

In [10]:
#combine the location to the dataframe
#load location data
locations =pd.read_csv('data_sets/sensor_locations.csv')

In [11]:
# setting the id to index and make it the columns
locations = locations.set_index(keys='id').transpose()

In [12]:
locations = pd.read_csv('data_sets/sensor_locations.csv')

In [13]:
locations = locations.set_index('id')

In [14]:
# creating a dictionary with all the sensors and times, change the columns to be the same, add latitude & long titude
d = {}
for num in to_add:
    d[f"sensor{num}"] = df[['UTC time',f'{num}_temperature',f'{num}_humidity',f'{num}_pressure',f'{num}_pm1',f'{num}_pm25',f'{num}_pm10']]
    d[f'sensor{num}'].columns = ['UTC time', 'temperature', 'humidity', 'pressure', 'pm1','pm25', 'pm10']
    d[f'sensor{num}']['latitude'] = locations.loc[int(num),'latitude']
    d[f'sensor{num}']['longitude'] = locations.loc[int(num),'longitude']

In [15]:
# adding the df together to create one long df
sensor_d_list = []
for num in to_add:
    sensor_d_list.append(d[f'sensor{num}'])
df_r = pd.concat(sensor_d_list)

In [16]:
# fill in the remaining columns with average & create dummies variables for lat and long, convert UTC to date time
# converting UTC to date_time
df_r['UTC time'] = pd.to_datetime(df_r['UTC time'])

# UTC time as the index
df_r.set_index(keys='UTC time', inplace=True)

In [17]:
# some of the humidity level are negative, this doesn't really make sense. Replace those with 1 
def replace_hum(hum):
    if hum < 1:
        return 1
    else:
        return hum

df_r['humidity'] = df_r['humidity'].apply(lambda x: replace_hum(x))

In [35]:
# filling in the remaining missing data with the average
for num in range(1,13):
    df_r.loc[f'2017-{num}'] = df_r.loc[f'2017-{num}'].apply(lambda x: x.fillna(x.mean()),axis=1)

In [51]:
df_r.columns

Index(['temperature', 'humidity', 'pressure', 'pm1', 'pm25', 'pm10',
       'latitude', 'longitude'],
      dtype='object')

In [55]:
# turning the latitude and longtitude in to dummies

df_clean = pd.get_dummies(data= df_r,columns = ['latitude','longitude'])

In [58]:
df_clean.to_csv('data_sets/data_clean.csv',index= False)

In [59]:
pd.read_csv('data_sets/data_clean.csv')

,temperature,humidity,pressure,pm1,pm25,pm10,latitude_49.99848,latitude_49.999615000000006,latitude_50.005587,latitude_50.013156,...,longitude_19.953954,longitude_19.962347,longitude_19.966073,longitude_19.968416,longitude_19.993572,longitude_20.007551,longitude_20.018922,longitude_20.027745,longitude_20.045773,longitude_20.101569
0,2.0,28.0,102212.0,89.0,98.0,156.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1.0,29.0,102160.0,91.0,100.0,161.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1.0,29.0,102085.0,101.0,113.0,181.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1.0,28.0,102038.0,100.0,112.0,178.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1.0,27.0,101983.0,73.0,78.0,129.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233420,7.0,85.0,101728.0,9.0,10.0,18.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
233421,6.0,85.0,101769.0,9.0,10.0,17.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
233422,6.0,83.0,101777.0,7.0,8.0,14.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
233423,6.0,85.0,101778.0,7.0,8.0,14.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
